In [39]:
from pathlib import Path
import asyncio
import json
import fsspec

from kerchunk.combine import MultiZarrToZarr, merge_vars
from kerchunk.zarr import ZarrToZarr

import xarray as xr
import pvdeg

In [40]:
pvdeg.geospatial.start_dask()

/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46245 instead
  warnings.warn(


Dashboard: http://127.0.0.1:46245/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46245/status,
Dashboard: http://127.0.0.1:46245/status,Workers: 16
Total threads: 128,Total memory: 503.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38301,Workers: 16
Dashboard: http://127.0.0.1:46245/status,Total threads: 128
Started: Just now,Total memory: 503.15 GiB
Comm: tcp://127.0.0.1:37467,Total threads: 8
Dashboard: http://127.0.0.1:38323/status,Memory: 31.45 GiB
Nanny: tcp://127.0.0.1:35587,


In [46]:
def find_valid_gids(ds):
    
    # Collapse across all data variables: build a mask where any NaN is True
    nan_mask = xr.zeros_like(ds["gid"], dtype=bool)  # start with False mask, shape (gid,)
    nan_mask = nan_mask.astype(bool)
    
    for v in ds.data_vars:
        # Reduce over all non-gid dimensions to see if this gid has any NaN in this variable
        has_nan = ds[v].isnull().any(dim=[d for d in ds[v].dims if d != "gid"])
        nan_mask = nan_mask | has_nan
    
    # gids that never had a NaN in any variable
    valid_gids = ds["gid"].where(~nan_mask, drop=True)

    return valid_gids

I see there's still an annual_energy and subarray1_dc_gross in the beta dataset, in addition to the 2 normalized versions. I would suggest dropping those unnormalized ones in the final, because I think we don't want to confuse people with the 80 kW options

In [20]:
model_outs_dir = Path("/projects/inspire/PySAM-MAPS/v0.5/colorado/")
post_outs_dir = Path("/projects/inspire/PySAM-MAPS/v0.5/postprocess/colorado")

model_outs_zarrs = sorted(list(model_outs_dir.glob("*.zarr")))
post_outs_zarrs = sorted(list(post_outs_dir.glob("*.zarr")))

refs = {}
for index, (model_out_path, post_out_path) in enumerate(zip(model_outs_zarrs, post_outs_zarrs)):
    ref_model =ZarrToZarr(str(model_out_path)).translate()
    ref_post = ZarrToZarr(str(post_out_path)).translate()

    ref_merged = merge_vars([ref_model, ref_post])
    refs[f"{index+1:02d}"] = ref_merged

In [49]:
for key, references_dict in refs.items():

    print("starting ", key)
    
    fs = fsspec.filesystem(
        "reference",
        fo=references_dict,
        remote_protocol="file",
        remote_options={}
    )
    
    mapper = fs.get_mapper("")
    
    ds = xr.open_dataset(mapper, engine="zarr", backend_kwargs={"consolidated": False})

    valid_gids = find_valid_gids(ds)

    ds.sel(gid=valid_gids).to_zarr(f"/projects/inspire/PySAM-MAPS/beta/{key}.zarr")

In [50]:
insp

xr.open_zarr("/projects/inspire/PySAM-MAPS/beta/01.zarr")

<xarray.Dataset> Size: 5GB
Dimensions:                                (gid: 2393, time: 8760, distance: 10)
Coordinates:
  * distance                               (distance) int64 80B 0 1 2 ... 7 8 9
  * gid                                    (gid) int64 19kB 398600 ... 560852
  * time                                   (time) datetime64[ns] 70kB 2001-01...
Data variables: (12/26)
    albedo                                 (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>
    annual_energy                          (gid) float64 19kB dask.array<chunksize=(100,), meta=np.ndarray>
    annual_energy_per_acre                 (gid) float64 19kB dask.array<chunksize=(100,), meta=np.ndarray>
    annual_energy_per_kWdc_installed       (gid) float64 19kB dask.array<chunksize=(100,), meta=np.ndarray>
    annual_poa                             (gid) float64 19kB dask.array<chunksize=(100,), meta=np.ndarray>
    beda                                   (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>
    ...                                     ...
    subarray1_poa_rear                     (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>
    temp_air                               (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>
    tilt                                   (gid) float64 19kB dask.array<chunksize=(100,), meta=np.ndarray>
    underpannel                            (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>
    wind_direction                         (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>
    wind_speed                             (gid, time) float64 168MB dask.array<chunksize=(100, 8760), meta=np.ndarray>

In [54]:
inspire_agrivolt.verify.gids_with_time_nans(
    xr.open_zarr("/projects/inspire/PySAM-MAPS/beta/03.zarr")
)


(array([], dtype=int64),
 {'albedo': 0,
  'beda': 0,
  'bedb': 0,
  'bedc': 0,
  'dhi': 0,
  'dni': 0,
  'edgetoedge': 0,
  'ghi': 0,
  'ground_irradiance': 0,
  'relative_humidity': 0,
  'subarray1_celltemp': 0,
  'subarray1_dc_gross': 0,
  'subarray1_dc_gross_per_acre': 0,
  'subarray1_dc_gross_per_kWdc_installed': 0,
  'subarray1_poa_front': 0,
  'subarray1_poa_rear': 0,
  'temp_air': 0,
  'underpannel': 0,
  'wind_direction': 0,
  'wind_speed': 0})